In [1]:
# from resnet_18_fcn import *
from resnet_34_fcn import *
import time
from torch.utils.data import DataLoader
import torch
import gc
import voc
import torchvision.transforms as standard_transforms
import util
import numpy as np
import sys
import math
import copy

class MaskToTensor(object):
    def __call__(self, img):
        return torch.from_numpy(np.array(img, dtype=np.int32)).long()

# Initialize Weights with Xavier Weight Initialization
def init_weights(m):
    if isinstance(m, nn.ConvTranspose2d):
        torch.nn.init.xavier_uniform_(m.weight.data)
#         if m.bias is not None:  # Check if the bias exists
        torch.nn.init.normal_(m.bias.data)  # Xavier not applicable for biases

In [2]:
mean_std = ([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
input_transform = standard_transforms.Compose([
        standard_transforms.ToTensor(),
        standard_transforms.Normalize(*mean_std)
    ])
target_transform = MaskToTensor()

train_dataset =voc.VOC('train', transform=input_transform, target_transform=target_transform)
val_dataset = voc.VOC('val', transform=input_transform, target_transform=target_transform)
test_dataset = voc.VOC('test', transform=input_transform, target_transform=target_transform)

train_loader = DataLoader(dataset=train_dataset, batch_size= 16, shuffle=True)
val_loader = DataLoader(dataset=val_dataset, batch_size= 16, shuffle=False)
test_loader = DataLoader(dataset=test_dataset, batch_size= 16, shuffle=False)

epochs = 20
n_class = 21

fcn_model = FCN_ResNet34(n_class=n_class)
fcn_model.apply(init_weights)

/opt/conda/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


FCN_ResNet34(
  (encoder): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=

In [3]:
device = torch.device("cuda") if torch.cuda.is_available() else "cpu"

optimizer = torch.optim.Adam(fcn_model.parameters(), lr=5e-4)

# Choose an appropriate loss function from https://pytorch.org/docs/stable/_modules/torch/nn/modules/loss.html
criterion = nn.CrossEntropyLoss()
fcn_model = fcn_model.to(device)
max_model = fcn_model

earlystop = 3

In [4]:
def train():
    """
    Train a deep learning model using mini-batches.

    - Perform forward propagation in each epoch.
    - Compute loss and conduct backpropagation.
    - Update model weights.
    - Evaluate model on validation set for mIoU score.
    - Save model state if mIoU score improves.
    - Implement early stopping if necessary.

    Returns:
        None.
    """

    patience = 0
    best_iou_score = 0.0
    losses = []

    for epoch in range(epochs):
        ts = time.time()
        for iter, (inputs, labels) in enumerate(train_loader):
            
            optimizer.zero_grad() # reset optimizer gradients

            inputs, labels = inputs.to(device), labels.to(device) # both inputs and labels in device as model

            outputs = fcn_model(inputs) #  Compute outputs. Automatically in the same device as the model's

            loss = criterion(outputs, labels) #Calculate loss

            loss.backward() # Bckpropagate model

            optimizer.step() # Update the weights
            
            losses.append(loss.item())

            #if iter % 10 == 0:
            #    print("epoch{}, iter{}, loss: {}".format(epoch, iter, loss.item()))

        print("Finish epoch {}, time elapsed {}".format(epoch, time.time() - ts))
        print("Train Avg Loss: {}".format(np.mean(losses)))

        current_miou_score = val(epoch)

        # Save current IoU if better than stored best
        if current_miou_score > best_iou_score:
            best_iou_score = current_miou_score
            patience = 0
            max_model = copy.deepcopy(fcn_model) # save the best model
        else:
            patience += 1
            
        # Early stop if patience level is met
        if patience >= earlystop:
            print("Early stop at epoch " + str(epoch))
            break
    

In [5]:
def val(epoch):
    """
    Validate the deep learning model on a validation dataset.

    - Set model to evaluation mode. DONE
    - Disable gradient calculations. DONE
    - Iterate over validation data loader:
        - Perform forward pass to get outputs.
        - Compute loss and accumulate it.
        - Calculate and accumulate mean Intersection over Union (IoU) scores and pixel accuracy.
    - Print average loss, IoU, and pixel accuracy for the epoch.
    - Switch model back to training mode.

    Args:
        epoch (int): The current epoch number.

    Returns:
        tuple: Mean IoU score and mean loss for this validation epoch.
    """
    fcn_model.eval() # Put in eval mode (disables batchnorm/dropout) !
    
    losses = []
    mean_iou_scores = []
    accuracy = []
    
    with torch.no_grad(): # we don't need to calculate the gradient in the validation/testing

        # Iterate through Validation Set
        for iter, (input, label) in enumerate(val_loader):
            # label = (16, 224, 224) / batch size 16 of 244*244 masks
            # output = (16, 21, 224, 224) / batch size 16 of 21 possible classes of 244*244 masks
            
            input, label = input.to(device), label.to(device) # both inputs and labels in device as model
            
            output = fcn_model.forward(input) # Perform forward pass to get outputs.
            N, numClass, H, W = output.shape

            prediction = output.view(N, n_class, -1).argmax(dim=1).view(N, H, W) # Find the prediction for each pixel
            
            loss = criterion(output, label) # Compute loss and accumulate it.
            losses.append(loss.item())
            
            meanIOU = util.iou(prediction, label, n_class) # Calculate Intersection over Union (IoU) scores
            mean_iou_scores.append(meanIOU)

            acc = util.pixel_acc(prediction, label) # Calculate pixel accuracy
            accuracy.append(acc)
    
    print(f"Validation Loss: {np.mean(losses)}")
    print(f"Validation IoU: {np.mean(mean_iou_scores)}")
    print(f"Validation Pixel Acc: {np.mean(accuracy)}")
    print("\n")

    fcn_model.train() #TURNING THE TRAIN MODE BACK ON TO ENABLE BATCHNORM/DROPOUT!!

    return np.mean(mean_iou_scores)

def modelTest():
    """
    Test the deep learning model using a test dataset.

    - Load the model with the best weights.
    - Set the model to evaluation mode.
    - Iterate over the test data loader:
        - Perform forward pass and compute loss.
        - Accumulate loss, IoU scores, and pixel accuracy.
    - Print average loss, IoU, and pixel accuracy for the test data.
    - Switch model back to training mode.

    Returns:
        None. Outputs average test metrics to the console.
    """

    fcn_model = copy.deepcopy(max_model) # Asssume model loaded with the best weights.
    
    fcn_model.eval()  # Put in eval mode (disables batchnorm/dropout) !

    losses = []
    mean_iou_scores = []
    accuracy = []

    with torch.no_grad():  # we don't need to calculate the gradient in the validation/testing

        # Iterate through Test Set
        for iter, (input, label) in enumerate(test_loader):

            input, label = input.to(device), label.to(device) # both inputs and labels in device as model

            output = fcn_model.forward(input) # Perform forward pass to get outputs.
            N, numClass, H, W = output.shape

            prediction = output.view(N, n_class, -1).argmax(dim=1).view(N, H, W) # Find the prediction for each pixel
            
            loss = criterion(output, label) # Compute loss and accumulate it.
            losses.append(loss.item())
            
            meanIOU = util.iou(prediction, label, n_class) # Calculate Intersection over Union (IoU) scores
            mean_iou_scores.append(meanIOU)

            acc = util.pixel_acc(prediction, label) # Calculate pixel accuracy
            accuracy.append(acc)

    print(f"Test Loss at Test: {np.mean(losses)}")
    print(f"Test IoU at Test: {np.mean(mean_iou_scores)}")
    print(f"Test Pixel acc at Test: {np.mean(accuracy)}")

    fcn_model.train()  #TURNING THE TRAIN MODE BACK ON TO ENABLE BATCHNORM/DROPOUT!!

In [6]:
def exportModel(inputs):    
    """
    Export the output of the model for given inputs.

    - Set the model to evaluation mode.
    - Load the model with the best saved weights.
    - Perform a forward pass with the model to get output.
    - Switch model back to training mode.

    Args:
        inputs: Input data to the model.

    Returns:
        Output from the model for the given inputs.
    """

    fcn_model.eval() # Put in eval mode (disables batchnorm/dropout) !
    
    saved_model_path = "Fill Path To Best Model"
    # TODO Then Load your best model using saved_model_path
    
    inputs = inputs.to(device)
    
    output_image = fcn_model(inputs)
    
    fcn_model.train()  #TURNING THE TRAIN MODE BACK ON TO ENABLE BATCHNORM/DROPOUT!!
    
    return output_image

# if __name__ == "__main__":

#     val(0)  # show the accuracy before training
#     train()
#     modelTest()

#     # housekeeping
#     gc.collect()
#     torch.cuda.empty_cache()

In [14]:
train()

Finish epoch 0, time elapsed 3.3556559085845947
Train Avg Loss: 0.9802679802690234
Validation Loss: 1.073997552905764
Validation IoU: 0.07851260735273782
Validation Pixel Acc: 0.7618555363691234


Finish epoch 1, time elapsed 3.2676329612731934
Train Avg Loss: 0.9238160508019584
Validation Loss: 1.0749568087714059
Validation IoU: 0.08399816998726871
Validation Pixel Acc: 0.7718467067351493


Finish epoch 2, time elapsed 3.333735942840576
Train Avg Loss: 0.9066412207626161
Validation Loss: 1.0440470959459032
Validation IoU: 0.0840849129705137
Validation Pixel Acc: 0.7710566484545829


Finish epoch 3, time elapsed 3.0020248889923096
Train Avg Loss: 0.8845059382064002
Validation Loss: 1.060536001409803
Validation IoU: 0.086545168833566
Validation Pixel Acc: 0.7715032905600856


Finish epoch 4, time elapsed 3.184597969055176
Train Avg Loss: 0.8872504958084652
Validation Loss: 1.010516575404576
Validation IoU: 0.08735910334565357
Validation Pixel Acc: 0.7756700521307854


Finish epoch 5, ti

In [15]:
modelTest()

Test Loss at Test: 1.0503004150731223
Test IoU at Test: 0.12382800403897062
Test Pixel acc at Test: 0.7688624434846484


## Q4.a COSINE ANNEALING LR

In [16]:
fcn_model = FCN_ResNet34(n_class=n_class)
fcn_model.apply(init_weights)
fcn_model = fcn_model.to(device)

earlystop = 3
max_model = fcn_model

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(fcn_model.parameters(), lr=1e-3)

#Test Cosine Annealing Learning Rate
iterMax = math.floor(len(train_dataset)/16)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer=optimizer, T_max=iterMax, eta_min=1e-4)

In [17]:
def train1():
    patience = 0
    best_iou_score = 0.0

    for epoch in range(epochs):
        losses = []
        ts = time.time()
        for iter, (inputs, labels) in enumerate(train_loader):
            
            optimizer.zero_grad() # reset optimizer gradients

            inputs, labels = inputs.to(device), labels.to(device) # both inputs and labels in device as model

            outputs = fcn_model(inputs) #  Compute outputs. Automatically in the same device as the model's

            loss = criterion(outputs, labels) #Calculate loss

            loss.backward() # Bckpropagate model

            optimizer.step() # Update the weights
            
            losses.append(loss.item())
            
            scheduler.step() # For cosine annealing learning rate

        print("Finish epoch {}, time elapsed {}".format(epoch, time.time() - ts))
        print("Train Avg Loss: {}".format(np.mean(losses)))
        
        current_miou_score = val(epoch)

        if current_miou_score > best_iou_score:
            best_iou_score = current_miou_score
            patience = 0
            max_model = copy.deepcopy(fcn_model)
            # save the best model
        else:
            patience += 1
            
        # Early stop if patience level is met
        if patience >= earlystop:
            print("Early stop at epoch " + str(epoch))
            break
    

In [18]:
train1()

Finish epoch 0, time elapsed 3.144139289855957
Train Avg Loss: 3.0407685382025584
Validation Loss: 2.9449727024350847
Validation IoU: 0.03033509839788575
Validation Pixel Acc: 0.3362581214126275


Finish epoch 1, time elapsed 3.3123090267181396
Train Avg Loss: 2.7771624497004916
Validation Loss: 2.9277971982955933
Validation IoU: 0.04333676457236222
Validation Pixel Acc: 0.6198120295132562


Finish epoch 2, time elapsed 3.1016955375671387
Train Avg Loss: 2.5145740679332187
Validation Loss: 2.1911818981170654
Validation IoU: 0.05286295347671104
Validation Pixel Acc: 0.7012355259486608


Finish epoch 3, time elapsed 3.246269464492798
Train Avg Loss: 2.3795946666172574
Validation Loss: 2.1329251868384227
Validation IoU: 0.054918439730762526
Validation Pixel Acc: 0.7084538140032798


Finish epoch 4, time elapsed 3.081167221069336
Train Avg Loss: 2.1382357137543813
Validation Loss: 2.013813410486494
Validation IoU: 0.057676971377836554
Validation Pixel Acc: 0.7169075746230412


Finish epoch

In [19]:
modelTest()

Test Loss at Test: 1.7479261926242284
Test IoU at Test: 0.06461913530705249
Test Pixel acc at Test: 0.7132075451553389


## Q4.b Image Transformation

In [20]:
fcn_model = FCN_ResNet34(n_class=n_class)
fcn_model.apply(init_weights)
fcn_model = fcn_model.to(device)

earlystop = 3
max_model = fcn_model

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(fcn_model.parameters(), lr=1e-3)
#Test Cosine Annealing Learning Rate
iterMax = math.floor(len(train_dataset)/16)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer=optimizer, T_max=iterMax, eta_min=1e-4)

In [21]:
train1()

Finish epoch 0, time elapsed 3.2206363677978516
Train Avg Loss: 2.8587160451071605
Validation Loss: 2.816209844180516
Validation IoU: 0.044183768605920805
Validation Pixel Acc: 0.5775929976482781


Finish epoch 1, time elapsed 3.0145652294158936
Train Avg Loss: 2.599846839904785
Validation Loss: 2.434306468282427
Validation IoU: 0.05095173401966833
Validation Pixel Acc: 0.6838541903926748


Finish epoch 2, time elapsed 3.1507327556610107
Train Avg Loss: 2.2725536482674733
Validation Loss: 2.0983338270868575
Validation IoU: 0.05592585548426308
Validation Pixel Acc: 0.7118182390841382


Finish epoch 3, time elapsed 3.2615063190460205
Train Avg Loss: 2.1737447551318576
Validation Loss: 2.2848498736109053
Validation IoU: 0.0520654131949068
Validation Pixel Acc: 0.6728932906170281


Finish epoch 4, time elapsed 3.2354679107666016
Train Avg Loss: 2.0296574405261447
Validation Loss: 1.8286787697247096
Validation IoU: 0.059507923008250495
Validation Pixel Acc: 0.7275374076109238


Finish epoch

In [22]:
modelTest()

Test Loss at Test: 1.8853135449545724
Test IoU at Test: 0.07405152428253936
Test Pixel acc at Test: 0.7016228767595208


## Q4.c Weight Imbalance + (Image Transformation)

In [27]:
def getClassWeights():
    ans = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    
    # Iterate through the training set
    for iter, (inputs, labels) in enumerate(train_loader):  
        unique_elements, counts = torch.unique(labels, return_counts=True)
        
        # Count number of each class
        for i in range(len(unique_elements)):
            ans[unique_elements[i]] += counts[i]

    normalized = [tensor.tolist() for tensor in ans]
    #normalized = [num/sum(normalized) for num in normalized]
    normalized = [1/(1-pow(0.1,num/50000)) for num in normalized]
    return torch.tensor(normalized)
#     class_counts = [0] * 21  # Assuming 21 classes
    
#     # Iterate through the training set
#     for _, labels in train_loader:
#         labels = labels.view(-1)  # Flatten labels tensor to 1D, if necessary
#         unique_elements, counts = torch.unique(labels, return_counts=True)
        
#         # Ensure unique_elements is a tensor of integers for indexing
#         unique_elements = unique_elements.to(torch.int64)
        
#         # Update counts for each class
#         for i in range(len(unique_elements)):
#             element = unique_elements[i].item()  # Convert to Python int
#             count = counts[i].item()
#             if element < len(class_counts):  # Check is now valid
#                 class_counts[element] += count
    
#     # Avoid division by zero for classes not present in the dataset
#     class_counts = [max(count, 1) for count in class_counts]

#     # Calculate weights inversely proportional to class frequencies
#     total_count = sum(class_counts)
#     weights = [total_count / count for count in class_counts]
    
#     # Normalize weights (optional)
#     min_weight = min(weights)
#     normalized_weights = [weight / min_weight for weight in weights]

#     return torch.tensor(normalized_weights, dtype=torch.float)

In [28]:
class_weights = getClassWeights()
print(class_weights)
class_weights = class_weights.to(device)

tensor([1.0000, 1.1215, 1.1613, 1.0169, 1.1303, 1.0395, 1.0001, 1.0340, 1.0002,
        1.0060, 1.0280, 1.0025, 1.0013, 1.0195, 1.0268, 1.0000, 1.3122, 1.5279,
        1.0004, 1.0041, 1.0025])


In [29]:
fcn_model = FCN_ResNet34(n_class=n_class)
fcn_model.apply(init_weights)
fcn_model = fcn_model.to(device)

earlystop = 5
max_model = fcn_model

optimizer = torch.optim.Adam(fcn_model.parameters(), lr=1e-2)
criterion = nn.CrossEntropyLoss(weight=class_weights)
#Test Cosine Annealing Learning Rate
# T_max = 10
iterMax = math.floor(len(train_dataset)/16)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer=optimizer, T_max=iterMax, eta_min=1e-4)
# scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=T_max, T_mult=1, eta_min=1e-4)

In [30]:
train1()

Finish epoch 0, time elapsed 3.1398379802703857
Train Avg Loss: 2.7007069247109547
Validation Loss: 5.171793631144932
Validation IoU: 0.03258818984972652
Validation Pixel Acc: 0.4684217625387208


Finish epoch 1, time elapsed 3.0589396953582764
Train Avg Loss: 2.0742288572447642
Validation Loss: 1.9567525471959795
Validation IoU: 0.06467935824710723
Validation Pixel Acc: 0.7250223676942875


Finish epoch 2, time elapsed 3.155501365661621
Train Avg Loss: 1.466308321271624
Validation Loss: 1.2651709020137787
Validation IoU: 0.06775255963608295
Validation Pixel Acc: 0.7602605080117985


Finish epoch 3, time elapsed 3.1819887161254883
Train Avg Loss: 1.300225555896759
Validation Loss: 1.1814778915473394
Validation IoU: 0.06587561035504445
Validation Pixel Acc: 0.7602094377790178


Finish epoch 4, time elapsed 3.2037124633789062
Train Avg Loss: 1.123200339930398
Validation Loss: 1.1234589091369085
Validation IoU: 0.07173672327246364
Validation Pixel Acc: 0.7627502975241437


Finish epoch 5,

In [31]:
modelTest()

Test Loss at Test: 1.1259074466569083
Test IoU at Test: 0.07159268437145956
Test Pixel acc at Test: 0.7408034433081269
